In [1]:
Packages <- c("dplyr","broom","geosphere","swfscMisc", "geosphere","MASS", "fields","lattice", "ggplot2", "bbmle", "ncdf4","dplyr","tidyr", "stringr", "tidyverse", "raster", "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")


In [ ]:
#read in data and add site names
conn_mat_all <- brick("~/oceanography/ROMS/data/Camotes_Sea_Connectivity_Matrices_08_day_PLD_integer.nc", values=T) 
#read in the data for the number of grid cells per site (site ids are North- South, as in, Palanas is number 1)
sites_n <- read.table("~/oceanography/ROMS/input//camotes_vertices_sites_results_water_only.txt", sep=" ", header=T, stringsAsFactors = F)
#the order of sites in the connectivity matrix
sites_order <- read.table("~/oceanography/ROMS/input/sites_order_as_in_roms.txt", sep=" ", header=T, stringsAsFactors = F)

sites <- as.character(c("Palanas", "Wangag", "Magbangon", "Cabatoan", "Caridad Cemetery", "Caridad Proper", "Hicgop", "Hicgop South", "Sitio Tugas", "Elementary School", "Sitio Lonas", "San Agustin", "Poroc San Flower", "Poroc Rose", "Pangasugan", "Visca", "Gabas", "Tamakin Dacot", "Haina", "Sitio Baybayon", "SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "CAI", "Other"))
site_ind <- c(seq(1,28, 1))
site_id <- as.data.frame(cbind(sites, site_ind), stringsAsFactors = F)
site_id$site_ind <- as.numeric(site_id$site_ind)


In [ ]:
#add in dates (seasons here are field seasons, not monsoon seasons)
col <- "date"
col2 <- "index"

season1 <- as.data.frame(seq(as.Date("2010/10/01"), as.Date("2011/05/31"), by="days"))
names(season1) <- col

season2 <- as.data.frame(seq(as.Date("2011/10/01"), as.Date("2012/05/31"), by="days"))
names(season2) <- col

season3 <- as.data.frame(seq(as.Date("2012/10/01"), as.Date("2013/05/31"), by="days"))
names(season3) <- col

season4 <- as.data.frame(seq(as.Date("2013/10/01"), as.Date("2014/04/18"), by="days"))
names(season4) <- col

all_dates <- bind_rows(season1, season2, season3, season4)
time_step <- as.data.frame(seq(1,930,1))
names(time_step)<- col2
all_dates2 <- bind_cols(time_step, all_dates)
dim(all_dates2)


In [ ]:
#make each daily connectivity matrix into a df and then pull them all together into a list
#conn_mat_list <- vector("list", nrow(all_dates2))

#date_df <- do.call("rbind", replicate(784, date, simplify = FALSE))

conn_mat_full <- as.data.frame(matrix(nrow=0, ncol=4))

cols_conn_mat <- c("date","sink", "source", "number")
cols_tmp <- c("sink", "source", "number")

names(conn_mat_full) <- cols_conn_mat

for(i in 1:nrow(all_dates2)){
    
    index_eval <- all_dates2$index[i]
    
    tmp <- as.data.frame(conn_mat_all[[i]], xy=T)   #pull out a data frame for each date
    names(tmp) <- cols_tmp
    
    date <- filter(all_dates2, index==index_eval)[2] #grab the date
    date_df <- do.call("rbind", replicate(784, date, simplify = FALSE)) #make a df of the date
    tmp2 <- bind_cols(date_df, tmp)

 
    conn_mat_full <- rbind(conn_mat_full, tmp2) 
}



In [ ]:
#connect site names
conn_mat_full2 <- left_join(conn_mat_full, site_id, by=c(source="site_ind")) %>%
    dplyr::select(-source) %>%
    rename(source="sites")
    
conn_mat_full3 <- left_join(conn_mat_full2, site_id, by=c(sink="site_ind")) %>%
    dplyr::select(-sink) %>%
    rename(destination="sites")

    

In [ ]:
#make all versions of Hicgop "Hicgop South" to match empirical data
conn_mat_full3 <- conn_mat_full3 %>%
    mutate(source=ifelse(source=="Hicgop", "Hicgop South", source)) %>%
    mutate(destination=ifelse(destination=="Hicgop", "Hicgop South", destination))

#mark which sources/destinations were sampled in real life
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other") #although we didn't sample the Camotes Islands, I'm going to leave it in the simulation results for now to use it in the GLM of ROMS model. It will be remove in filtering to produce the df that will be used for comparisons to genetic observations
#mark the monsoon seasons in which they dispersed, based on the same criteria I used for the parentage indirectly through the growth estimates
NEM <- c(11, 12, 1, 2, 3, 4, 5, 6)
SWM <- c(7, 8, 9, 10)

#add a column showing which sources and destinations were empirically sampled, and corresponding date/year/season data
conn_mat_full4 <- conn_mat_full3 %>%
    mutate(SourceSampled= ifelse(source %!in% UnsampledSites, "yes", "no")) %>%
    mutate(DestSampled= ifelse(destination %!in% UnsampledSites, "yes", "no")) %>%
    mutate(SimMonth=month(ymd(date))) %>%
    mutate(SimDay=month(ymd(date))) %>%
    mutate(SimYear=year(ymd(date))) %>%
    mutate(YearSampled=ifelse(date %in% season1$date, "2011", ifelse(date %in% season2$date, "2012", ifelse(date %in% season3$date, "2013", "2014")))) %>%
    mutate(YearSampled=as.factor(YearSampled)) %>%
    mutate(SimMonsoon=ifelse(SimMonth %in% NEM, "NEM", "SWM")) %>%
    rename(DailyParticles="number") %>%
    relocate(DailyParticles,.after=SimMonsoon)
    
#note to self, I think I messed up sources/destinations in the orginal for loop!!!! so I'm grouping by source below, but that's because the true destination column is misnamed "source" in the code and I haven't figured it out yet
#!!!!
 

__Skip above and read the tidied data frame below__

__Add in genetic sampling data__

In [2]:
#head(conn_mat_full4)
#write.csv(conn_mat_full4, file="~/oceanography/script_output/SimConnectivityTable.csv", row.names=F)
SimConnDF <- read.csv(file="~/oceanography/script_output/SimConnectivityTable.csv", header=T) %>%
    mutate(YearSampled=as.character(YearSampled))
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other") #although we didn't sample the Camotes Islands, I'm going to leave it in the simulation results for now to use it in the GLM of ROMS model. It will be remove in filtering to produce the df that will be used for comparisons to genetic observations


In [3]:
#add together all of the particles for each season. At some point need to consider matching times that we sampled, from the fish meta-data?
#replace XN with years
SimConnDF <- SimConnDF %>%
    group_by(destination, YearSampled) %>%
    mutate(AnnTotalRec=sum(DailyParticles)) %>% #Annual here means the year that would be considered one year in the genetic analysis
    mutate(AnnTotalRecFromSampSites=ifelse(DestSampled=="yes" & SourceSampled=="yes",sum(DailyParticles), 0)) #this comes up at NA otherwise...maybe that's important for the GLM though. Expect the unsampled sites won't be used for the GLM. think more about this.



In [ ]:
#check that everything looks correct
test <- SimConnDF %>% filter(DailyParticles >1 & 
                             SourceSampled=="yes" & DestSampled=="yes")
#pdf("~/oceanography/script_output/HistNumParticlesRecruiting.pdf")
hist(test$DailyParticles, breaks=50, xlab="Number of particles recruiting daily", main=NULL)
#dev.off()


In [4]:
##put in North-South order for kernel fitting, also used for GLM connectivity data frame?
SitesNS <- read.csv("~/parentage/text_file/sites_NS.txt", stringsAsFactors = F) %>%
    mutate(site=ifelse(site =="S. Magbangon" , "Magbangon", site)) %>%
    filter(site != "N. Magbangon") 

###add site size - is this worth adding? It would be a random effect, I already know that it affects the number of particles...
##Reef_sizes <- read.csv("~/parentage/kernel_fitting/894_loci/area_unsurveyed.", header=FALSE)

#add the distances between sites, calculated from the site centroids
centroidsSamp <- read.csv("~/parentage/kernel_fitting/1340_loci/input/site_centroids.csv", header=TRUE, stringsAsFactors = F) %>%
    mutate(site=ifelse(site =="S.Magbangon" , "Magbangon", site)) %>%
    filter(site != "N.Magbangon") %>%
    filter(site %in% SitesNS$site)

#I want to average the Camotes Islands values and add those in. That way I can use that distance in the GLM of the simulated ROMS as response variable. this can be subsetted out for the observed data by using the "source/dests sampled" columns
#pull out the camotes centroids for a df with just them, to average lat/lon points with (centroids)
CAI <- c("camotes_central1", "camotes_central2", "camotes_east1", "camotes_east2", "camotes_west1", "camotes_west2")

centroidsSim <- read.csv("~/parentage/kernel_fitting/1340_loci/input/site_centroids.csv", header=TRUE, stringsAsFactors =F)
CamotesAvg <- centroidsSim %>%
    filter(site %in% CAI) %>%
    mutate(lon=mean(lon)) %>% #take the average lon
    mutate(lat=mean(lat)) %>% #take the average lat
    slice_head()

CamotesAvg$site <- "CAI"#make a one row DF for this value that can be added to centroids so it's distances are calculated

#add to tidy distance
centroids <- bind_rows(centroidsSamp, CamotesAvg) 

##calculate the distance from all potential parents and all potential offspring
all_possible_dists <- as.data.frame(rdist.earth(as.matrix(centroids[,c('lon', 'lat')]), as.matrix(centroids[,c('lon', 'lat')]), miles=FALSE, R=6371))

##attach the sample_ids to each distance, so you can also get site and year
colnames(all_possible_dists) <- centroids$site
all_possible_dists$site_i <- centroids$site

##gather into tidy df
all_possible_dists_tidy <- all_possible_dists %>%
    dplyr::select(site_i, everything()) %>%
    gather(2:20, key=site_j, value=dist_km) 
nrow(all_possible_dists_tidy)

SimConnDFDist <- left_join(SimConnDF, all_possible_dists_tidy, by=c(source="site_i", destination="site_j"))



[1] 361

In [6]:
#add a continuous value for direction in the form of a bearing
#col <- c("site_i", "site_j", "bearing")
AllBearings <- matrix(nrow=0, ncol=3)
#colnames(AllBearings) <- col

for(i in 1:nrow(centroids)) {
    
    site_i <- centroids[i,] #take the site name
    site_i_coords <- as.matrix(site_i[,2:3]) #get the lon/lat

    #now compare this site to all other sites for bearing
    
    bearings <- matrix(nrow=nrow(centroids), ncol=3)

    for(j in 1:nrow(centroids)){
        


        site_j <- centroids[j,]
        site_j_coords <- as.matrix(site_j[,2:3]) #get the lon/lat

        bearing <- geosphere::bearing(site_i_coords, site_j_coords)
        bearings[j,1] <- site_i[1,1] #add to data frame
        bearings[j,2] <- site_j[1,1]
        bearings[j,3] <- bearing

    }
    AllBearings <- rbind(AllBearings, bearings)

}
nrow(AllBearings)#should be 361
nrow(centroids)
19*19

col <- c("site_i", "site_j", "bearing")
AllBearings <- as.data.frame(AllBearings)
colnames(AllBearings) <- col
#write.csv(AllBearings, file="~/oceanography/script_output/SurveyData/AllBearings.csv", row.names=F)

SimConnDFDir <- left_join(SimConnDFDist, AllBearings, by=c(source="site_i", destination="site_j"))

[1] 361

[1] 19

[1] 361

In [9]:

#add how well we sampled sites- proportion of habitat sampled
load("~/parentage/r_data/cumulative_prop_hab_sampled_by_site.RData")
PropSamp <- cumulative_prop_hab_sampled_by_site %>%
    mutate(total_possible_sample_anems = ifelse(site=="Caridad Proper", 4, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Caridad Proper" & total_anems_sampled==4, 1, total_prop_hab_sampled_anems_tidied) ) %>%
    mutate(total_possible_sample_anems = ifelse(site=="Sitio Lonas", total_anems_sampled, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Sitio Lonas", 1, total_prop_hab_sampled_anems_tidied) )%>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(is.nan(total_prop_hab_sampled_anems_tidied), 0, total_prop_hab_sampled_anems_tidied) ) %>%
    dplyr::select(site, time_frame, end_year, total_prop_hab_sampled_anems_tidied) 

PropSamp$site <- gsub(". ", ".", PropSamp$site, fixed=TRUE) #fix spaces in Magbangon names

S.Mag <- PropSamp %>%#make a table for all of the S. Mabangon years/prop sampled. Then join to the DF, make a column adding S.Magbangon values to all rows of prop sampled, but only sub that value in for PropSamp in N.Magbangon rows, then rename N.Magbangon as Magbangon
    filter(site=="S.Magbangon") %>%
    dplyr::select(site, end_year, total_prop_hab_sampled_anems_tidied) %>%
    rename(S.MagVal="total_prop_hab_sampled_anems_tidied", extra="site")

PropSamp <- left_join(PropSamp, S.Mag, by=c("end_year")) %>%
    mutate(S.MagSum=total_prop_hab_sampled_anems_tidied+S.MagVal) %>% #create col adding the S.Mag values to prop hab - keep in mind values could be greater than 1- if so change them to 1
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="N.Magbangon", S.MagSum, total_prop_hab_sampled_anems_tidied)) %>%#sub this value in for only N.Mag
    mutate(site=ifelse(site=="N.Magbangon", "Magbangon", site)) %>%#change N.Mag name to generic Mag
    filter(site !="S.Magbangon") %>%#eliminate the S.Mag rows, they are now repeats
    dplyr::select(-S.MagVal,-extra, -S.MagSum, -time_frame) %>%
    rename(PropSamp="total_prop_hab_sampled_anems_tidied")


#check if there are values>1, should be none
sum(which(PropSamp$total_prop_hab_sampled_anems_tidied >1)) #zero, that's good

#first, summarize the simulated data to the level of a sampling year, remember, this data frame DOES NOT include unsampled sites at the moment
SimConnDFDist <- SimConnDFDist %>%
    ungroup() %>%
    group_by(source, destination, YearSampled) %>%
    mutate(AnnRecPart=sum(DailyParticles)) %>% # this is a really confusing column name, so drop the similar ones from earlier that are for the purposes of figuring out the unassigned juveniles for kernel fitting
    ungroup() %>%
    dplyr::select(-AnnTotalRec, -AnnTotalRecFromSampSites) #yikes figure out how to rename these confusing as is...
#last piece to explore ROMS model GLM- the Camotes don't fit into the North/South paradigm I have going, so manually say the direction is 
SimConnDFProp <- left_join(SimConnDFDist, PropSamp, by=c(source="site", YearSampled="end_year")) %>%#use end year here, remembering that it's cumulative prop hab sampled that is relevant to the parentage analysis
    rename(SourcePropSamp="PropSamp")
SimConnDFMeta <- left_join(SimConnDFProp, PropSamp, by=c(destination="site", YearSampled="end_year")) %>% #use end year here, remembering that it's cumulative prop hab sampled that is relevant to the parentage analysis
    rename(DestPropSamp="PropSamp") %>%
    ungroup() %>% #ungroup by year, otherwise grouping will override year filtering
    filter(YearSampled %in% c("2012", "2013", "2014")) %>% #remove 2011 simulation results, we didn't sampled that year    
    filter(source %!in% UnsampledSites & source !="CAI") %>% #we narrow down to only sampled sites at this point, because this is the data frame that will be joined to the observed data
    filter(destination%!in% UnsampledSites & destination!= "CAI") 


[1] 0

__CELL BELOW IS TESTING ADDING SEASON DATA, IGNORE UNTIL READY TO GO BACK__

In [ ]:
####this cell is a mess, trying to make a df with a row for every year/season/route combination, come back if I can after I make other progress
#ObsDisp <- read.csv(file="~/parentage/colony2/20200605_1340loci/results/20200624colony_dispersaldirection.csv", header=T, stringsAsFactors=F)
##
#ObsDisp$offs_site <- gsub(". ", ".", ObsDisp$offs_site, fixed=TRUE) #fix spaces in Magbangon names
#ObsDisp$par_site <- gsub(". ", ".", ObsDisp$par_site, fixed=TRUE) #fix spaces in Magbangon names
#
##add which monsoon season each recruit is presumed to have dispersed in
##read in the data on the presumed mosoon season recruits' dispersal (includes all recruits sampled)
#RecMonsoon <- read.csv(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon.csv",stringsAsFactors = F, header=TRUE)
#RecMonsoonPar <- RecMonsoon %>%
#    filter(matched_offs=="Y") %>%
#    dplyr::select(fish_indiv, season) #%>% #when I tested joining this table to ObsDisp, I selected year, year_match, too so I could double check the years were correct. They were, taking out that line of code now.
#    
#nrow(RecMonsoonPar)
##cross check that 46 is the correct number from another df created in the seasonal kernel fitting script
#RecMonsoonCheck <- read.csv(file="~/parentage/kernel_fitting/1340_loci/input/20200624_AllSeasonParentage.csv", stringsAsFactors = F, header=TRUE)
#nrow(RecMonsoonCheck) #should be 46
#
##use the ROMS simulated data frame to pull out all year and season combos to join with the recruit seasonality data frame, that way all possible observations of route/season/year combos will be included
#AllSeasons <- SimConnDFMeta %>%
#    dplyr::select(source, destination, YearSampled, SimMonsoon)
###add to ObsDisp
#
#ObsDispSeas <- left_join(ObsDisp, RecMonsoonPar, by=c(offs_fish_indiv="fish_indiv")) %>%
#    mutate(year=as.character(year))
#
##just like in PropSamp, combine Magbangons to match simulation
###for the observed parentage, there's only one case of a disp obs to/from the same site in the same year from both N and S Mag. So that's the only obs to combine, do it manually
#ObsDispAnn <- ObsDisp %>%
#    filter(year %in% c("2012", "2013", "2014")) %>% #, "2014"  #only use the data from the years that overlap between simulations and empirical sampling
#    dplyr::select(year, dist_par_km, offs_site, par_site) %>%
#    #mutate(seasonInterm=as.numeric(ifelse(is.na(season), 0, ifelse(season=="SWM", -1, ifelse(season=="NEM", 1, season))))) %>%
#    group_by(offs_site, par_site, year) %>%
#    #mutate(NumericSeason=sum(seasonInterm))%>%
#    group_by(offs_site, par_site, year) %>% #for seasonal, also group by season, after filtering out NA rows
#    mutate(obs_disp=n()) %>%
#    distinct(offs_site, par_site, year, .keep_all = TRUE) %>% #, season
#    mutate(obs_disp=ifelse(offs_site=="N.Magbangon" & par_site=="Palanas" & year=="2013", 2, obs_disp)) %>% #manually change the 1 to 2 to combine N/S Mag
#    filter(par_site !="S.Magbangon" & offs_site !="Palanas") %>% #drop the S.Mag obs so we don't double count
#    mutate(offs_site=ifelse(offs_site=="N.Magbangon", "Magbangon", offs_site)) %>%#change N.Mag name to generic Mag
#    mutate(par_site=ifelse(par_site=="N.Magbangon", "Magbangon", par_site)) %>%#change N.Mag name to generic Mag  
#    mutate(offs_site=ifelse(offs_site=="S.Magbangon", "Magbangon", offs_site)) %>%#change S.Mag name to generic Mag
#    mutate(par_site=ifelse(par_site=="S.Magbangon", "Magbangon", par_site)) %>% #change S.Mag name to generic Mag    
#    mutate(year=as.character(year)) %>%
#    ungroup() 
# 
#
##first, summarize the simulated data to the level of a sampling year and remove duplicates (daily rows, now that it's already summarized)
#SimConnDFMetaAnn <- SimConnDFMeta %>%
#    ungroup() %>%
#    #distinct(YearSampled, source, destination, .keep_all = T) %>%
#    dplyr::select(-date)
#GenSimConn <- left_join(SimConnDFMetaAnn, ObsDispAnn, by=c(source="par_site", destination="offs_site", YearSampled="year"))%>%
#    mutate(obs_disp=ifelse(is.na(obs_disp), 0, obs_disp)) #%>%
#    #dplyr::select(-SimMonsoon) #this doesn't have meaning for the GLM, because we summarised to annual data
#sum(GenSimConn$obs_disp)#should be 24
##add a column for binomial 1/0 if dispersal occured along that route
##GenSimConn <- GenSimConn %>%
##    mutate(BiDisp=ifelse(obs_disp >= 1, 1, obs_disp))
##sum(GenSimConn$BiDisp) #should be 20
##write out the table with all of the genetic sampling and simulated data
##write.csv(GenSimConn, file="~/oceanography/script_output/GenSimConnectivityFullTable.csv", row.names=F)
#

__CELL BELOW IS WORKING FOR GLM OF OBSERVED DATA BY SIMULATED DATA__

In [ ]:
##add in the empirical dispersal data from the genetics
ObsDisp <- read.csv(file="~/parentage/colony2/20200605_1340loci/results/20200624colony_dispersaldirection.csv", header=T, stringsAsFactors=F)
#
ObsDisp$offs_site <- gsub(". ", ".", ObsDisp$offs_site, fixed=TRUE) #fix spaces in Magbangon names
ObsDisp$par_site <- gsub(". ", ".", ObsDisp$par_site, fixed=TRUE) #fix spaces in Magbangon names

##just like in PropSamp, combine Magbangons to match simulation
###for the observed parentage, there's only one case of a disp obs to/from the same site in the same year from both N and S Mag. So that's the only obs to combine, do it manually
ObsDisp <- ObsDisp %>%
    filter(year %in% c("2012", "2013", "2014")) %>% #, "2014"  #only use the data from the years that overlap between simulations and empirical sampling
    dplyr::select(year, dist_par_km, offs_site, par_site) %>%
    group_by(offs_site, par_site, year) %>%
    summarise(obs_disp=n()) %>%
    mutate(obs_disp=ifelse(offs_site=="N.Magbangon" & par_site=="Palanas" & year=="2013", 2, obs_disp)) %>% #manually change the 1 to 2 to combine N/S Mag
    filter(par_site !="S.Magbangon" & offs_site !="Palanas") %>% #drop the S.Mag obs so we don't double count
    mutate(offs_site=ifelse(offs_site=="N.Magbangon", "Magbangon", offs_site)) %>%#change N.Mag name to generic Mag
    mutate(par_site=ifelse(par_site=="N.Magbangon", "Magbangon", par_site)) %>%#change N.Mag name to generic Mag  
    mutate(offs_site=ifelse(offs_site=="S.Magbangon", "Magbangon", offs_site)) %>%#change S.Mag name to generic Mag
    mutate(par_site=ifelse(par_site=="S.Magbangon", "Magbangon", par_site)) %>% #change S.Mag name to generic Mag    
    mutate(year=as.character(year)) %>%
    ungroup() 

#first, summarize the simulated data to the level of a sampling year and remove duplicates (daily rows, now that it's already summarized)
SimConnDFMetaAnn <- SimConnDFMeta %>%
    ungroup() %>%
    distinct(YearSampled, source, destination, .keep_all = T) %>%
    dplyr::select(-date)
GenSimConn <- left_join(SimConnDFMetaAnn, ObsDisp, by=c(source="par_site", destination="offs_site", YearSampled="year"))%>%
    mutate(obs_disp=ifelse(is.na(obs_disp), 0, obs_disp))
sum(GenSimConn$obs_disp)
#add a column for binomial 1/0 if dispersal occured along that route

#write out the table with all of the genetic sampling and simulated data
#write.csv(GenSimConn, file="~/oceanography/script_output/GenSimConnectivityFullTable.csv", row.names=F)


In [ ]:
head(GenSimConn)

__run GLM observed genetic dispersal predicted by simulations__

In [ ]:
mean(GenSimConn$obs_disp) #should be 0.024
var(GenSimConn$obs_disp) #should be 0.054
#overdispersion?
#no, because residual deviance of the model is not greater than the df
#Residual deviance:  38.221  on 948  degrees of freedom

In [ ]:
#plan:
#fit 3 successive log-linear models starting with main efects only(fit1), main effects plus all 2-way interactions (fit2), and added in all 3 way interactions (fit3)


In [ ]:
NullMod <- glm(obs_disp ~ dist_km , data=GenSimConn, family="poisson")
summary(NullMod)

In [ ]:
#only main effects, first find worthwhile predictors with AIC
#the intercept is very significant but I think that just means 0s are more common in the data. totally true!

ObsModExpYearFit1 <- glm(obs_disp ~ AnnRecPart + YearSampled + direction + dist_km +SourcePropSamp +DestPropSamp, data=GenSimConn, family="poisson")
MainEff_AnnRecPart <- update(ObsModExpYearFit1, . ~ . -AnnRecPart)
MainEff_YearSampled <- update(ObsModExpYearFit1, . ~ . -YearSampled)
MainEff_direction <- update(ObsModExpYearFit1, . ~ . -direction)
MainEff_dist_km <- update(ObsModExpYearFit1, . ~ . -dist_km)
MainEff_SourcePropSamp <- update(ObsModExpYearFit1, . ~ . -SourcePropSamp)
MainEff_DestPropSamp <- update(ObsModExpYearFit1, . ~ . -DestPropSamp)
#drop the 3 terms AIC doesn't support? maybe not though, because when there's an interaction that's the best model
MainEff_ARP_DPrp_Dist <- glm(obs_disp ~ YearSampled + direction +SourcePropSamp, data=GenSimConn, family="poisson")
#add the interaction between year and simulations
ObsModExpYearFit2Int <- glm(obs_disp ~ AnnRecPart*YearSampled + direction+SourcePropSamp, data=GenSimConn, family="poisson")
ObsModExpYearFit3Int <- glm(obs_disp ~ AnnRecPart*YearSampled*direction+SourcePropSamp, data=GenSimConn, family="poisson")

MainEffAIC <- as.data.frame(AIC(NullMod,  ObsModExpYearFit1, MainEff_AnnRecPart, MainEff_YearSampled, MainEff_direction, MainEff_dist_km, MainEff_SourcePropSamp, MainEff_DestPropSamp, ObsModExpYearFit2Int, ObsModExpYearFit3Int))
MainEffAIC$model <- row.names(MainEffAIC)
MainEffAIC <- MainEffAIC %>% arrange(AIC)
MainEffAIC
#write.csv(MainEffAIC, file="~/oceanography/script_output/GLM/GLMObsDispAIC.csv",row.names=F,  quote=F)


In [ ]:
summary(ObsModExpYearFit2Int ) #judging by the significant intercept (which should be AnnRecPart:Year2012?) when there's more particles expected from the ROMS model, there's actually less in the observed data.
ObsModExpYearFit2Int_Sum <- tidy(ObsModExpYearFit2Int)
#write.csv(ObsModExpYearFit2Int_Sum, file="~/oceanography/script_output/GLM/YearDispIntBestModSum.csv",row.names=F,  quote=F)


In [ ]:
head(SimConnDFDist)

In [ ]:
#this isn't significant, which means the residual difference is small enough here to say the model fits reasonably well
with(test_mod, cbind(res.deviance = deviance, df = df.residual,
  p = pchisq(deviance, df.residual, lower.tail=FALSE)))


In [ ]:
#how much devience is explained here?

In [ ]:
plot(obs_disp ~ ExpDisp, data=SimPlusGenWithDist)

In [ ]:
#plot(test_mod_int)

In [ ]:
#plot predicted model
range(SimPlusGenWithDist$ExpDisp)

In [ ]:
exp_disp_range <- seq(0, 66114, 1)

In [ ]:
obs_disp_pred <- predict(test_mod_int,type="response")

In [ ]:
plot( SimPlusGenWithDist$ExpDisp, SimPlusGenWithDist$obs_disp, pch = 16)#, xlab = "WEIGHT (g)", ylab = "VS")

lines(obs_disp_pred)

In [ ]:
#sources are rows, columns are destinations, checked this? check again
#also check if it matters that these are in alphabetical order YES it should be alphabetical
SimDisp2013Mat <- SimDisp2013 %>%
    group_by(destination, source) %>%
    spread(destination, SumParticles)
#
rownames(SimDisp2013Mat) <- SimDisp2013Mat$source
SimDisp2013Mat$source <- NULL

In [ ]:
#add in unassigned 
RowtoAddInt <- SimDisp2013RecSamp %>%
    ungroup() %>%
    arrange(destination) %>%
    dplyr::select(TotalUnassigned)

RowtoAdd <- as.data.frame(t(RowtoAddInt))
colnames(RowtoAdd) <- colnames(SimDisp2013Mat)

FullSimDisp2013 <- rbind(SimDisp2013Mat, RowtoAdd)
rownames(FullSimDisp2013) <- NULL

In [ ]:
FullSimDisp2012

In [ ]:
SimDisp2012Mat

In [ ]:
#turn into parentage matrix

#SimDisp2012Mat <- SimDisp2012 %>%
#    ungroup() %>%
#    spread(source, destination, SumParticles)
    

#make the rest of the kernel input files
Assignments <- FullSimDisp2013
Adult_sample_proportions <- matrix(nrow=18, ncol=1, 1) #we sampled all sites completely because we are working with simulated data
Sampled_reefs <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/site_index_all.csv", header=FALSE))[,18] #sampled all reefs in the distance matrix/area/centroid files
Distances <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/distance_matrix_unsurveyed.csv", header=FALSE))[-2, -2]  #drop a magbangon row  
Reef_sizes <- as.matrix(read.csv("~/parentage/kernel_fitting/894_loci/area_unsurveyed.csv", header=FALSE))[]
Centroids <- as.matrix(read.csv("~/parentage/kernel_fitting/894_loci/centroids_unsurveyed.csv", header=T))



In [ ]:
a=-10
b=10


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
Sim2013Fit

#doen't look right. Maybe fix theta, optimize k

In [ ]:
#monsoon season prep
NEM_months <- c(11, 12, 1, 2, 3, 4)
SWM_months <- c(5, 6, 7, 8, 9, 10)

NEM <- conn_mat_full4 %>%
    filter(month %in% NEM_months) %>%
    group_by(source, destination) %>%
    summarise(conn=max(fraction))

SWM <- conn_mat_full4 %>%
    filter(month %in% SWM_months) %>%
    group_by(source, destination) %>%
    summarise(conn=max(fraction))